In [6]:
import pandas as pd
import numpy as np
import scipy.optimize
from scipy.optimize import minimize
import statsmodels.api as sm

In [7]:
def predPCD(E_scale, S_scale):
    pcd = np.exp(E_scale + S_scale) + 4.42 
    return pcd

In [9]:
# ttdf is the old version of the Final_Translating_Time data without the combined regions
#ttdf = pd.read_csv('Final_Translating_Time_DataSet.csv')
#ttdf

ttdfV2 = pd.read_csv('Updated_03.2024_Final_Translating_Time_DataSet.csv')
ttdfV2

,Event,Event_Score,rat_PCD_pred,pig_PCD_pred,primate_PCD_pred,infant_PCD_pred
0,cranial motor nuclei-peak,0.903000,10.469647,25.623223,27.943502,34.474127
1,retinal ganglion cell generation-start,1.023000,11.240958,28.326567,30.942674,38.305934
2,subplate-start,1.038000,11.344044,28.687868,31.343513,38.818054
3,locus coeruleus - peak,1.070000,11.569197,29.476999,32.218999,39.936593
4,inferior olivary nucleus - peak,1.073000,11.590676,29.552283,32.302521,40.043303
...,...,...,...,...,...,...
109,Combined_Cortex_Inflection,2.418348,31.951978,100.915981,111.475584,141.196495
110,Midbrain_Inflection,2.427603,32.207974,101.813212,112.470999,142.468260
111,Right_Cortex_Inflection,2.446176,32.728894,103.638969,114.496550,145.056147
112,Left_Superior_Colliculus_Inflection,2.527348,35.122627,112.028691,123.804363,156.948007


In [25]:
#inf_MWF is the old version of the Infant MWF data with the previous event scores that were calculated
#inf_MWF = pd.read_excel('Infant_MWF_data.xlsx')
#inf_MWF

,Brain_Region,Event_Score,Inflection_Point
0,iLCaud_inflection,2.243079,813.974367
1,iRCaud_inflection,2.361196,910.077145
2,iLHippo_inflection,2.413815,907.913727
3,iRHippo_inflection,2.409848,943.751872
4,iThal_inflection,2.217468,632.471646


In [26]:
inf_MWF['infant_PCD_pred'] = inf_MWF['Event_Score'].apply(lambda e: predPCD(e, 2.5))
inf_MWF

,Brain_Region,Event_Score,Inflection_Point,infant_PCD_pred
0,iLCaud_inflection,2.243079,813.974367,119.207040
1,iRCaud_inflection,2.361196,910.077145,133.598642
2,iLHippo_inflection,2.413815,907.913727,140.577928
3,iRHippo_inflection,2.409848,943.751872,140.038787
4,iThal_inflection,2.217468,632.471646,116.304588


In [13]:
K = -4.42
human_SS = 2.5
def calcResidual(white_matter_parameter, e_score, PCD_day):
    pred_PCD_day = np.exp(e_score + 2.5 + white_matter_parameter) + K
    residuals = PCD_day - pred_PCD_day
    X = sm.add_constant(pred_PCD_day)
    model = sm.OLS(PCD_day, X).fit()
    influence = model.get_influence()
    leverage = influence.hat_matrix_diag
    return residuals, leverage

def loss_function(white_matter_parameter, e_score, PCD_day):
    residuals, leverage = calcResidual(white_matter_parameter, e_score, PCD_day)
    RSE = (np.sum(residuals ** 2) / len(e_score) - 2) ** (.5)
    standardized_residuals = residuals / (RSE * ((1-leverage) ** (.5)))
    return np.sum((residuals) / (standardized_residuals / (len(e_score) ** (.5))))

In [27]:
initial_guess = [0]
e_score = inf_MWF['Event_Score'].values
days_of_event = inf_MWF['Inflection_Point'].values

result = minimize(loss_function, initial_guess, args = (e_score, days_of_event))
result


  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 529.2093542741889
        x: [ 1.911e+00]
      nit: 6
      jac: [ 7.629e-06]
 hess_inv: [[ 8.685e-08]]
     nfev: 54
     njev: 27

In [28]:
MWF_bias = result.x[0]
MWF_bias

1.9110040356344982

In [29]:
def MWF_predPCD(E_scale, S_scale, MWF_bias):
    pcd = np.exp((E_scale + S_scale) + MWF_bias) + 4.42 
    return pcd

In [30]:
inf_MWF['adj_infant_PCD_pred'] = inf_MWF['Event_Score'].apply(lambda e: MWF_predPCD(e, 2.5, MWF_bias))
inf_MWF

,Brain_Region,Event_Score,Inflection_Point,infant_PCD_pred,adj_infant_PCD_pred
0,iLCaud_inflection,2.243079,813.974367,119.207040,780.365762
1,iRCaud_inflection,2.361196,910.077145,133.598642,877.651157
2,iLHippo_inflection,2.413815,907.913727,140.577928,924.830241
3,iRHippo_inflection,2.409848,943.751872,140.038787,921.185713
4,iThal_inflection,2.217468,632.471646,116.304588,760.745555
